# Multiple regression workbook

Tutorial: [Multivariate regression tutorial: aragonite saturation state](#Multivariate-regression-tutorial:-aragonite-saturation-state)

Exercises: [Application of final model for aragonite saturation state](Application-of-final-model-for-aragonite-saturation-state)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pingouin as pg

import PyCO2SYS as pyco2
import seaborn as sns

In [3]:
filename07 = 'wcoa_cruise_2007/32WC20070511.exc.csv'
df07 = pd.read_csv(filename07,header=29,na_values=-999,
                  parse_dates=[[6,7]])

In [4]:
filename13 = 'wcoa_cruise_2013/WCOA2013_hy1.csv'
df13 =  pd.read_csv(filename13,header=31,na_values=-999,parse_dates=[[8,9]])

/var/folders/z7/lmyk7sz94177j166ck0x63h80000gr/T/ipykernel_26314/3033169421.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df13 =  pd.read_csv(filename13,header=31,na_values=-999,parse_dates=[[8,9]])


In [5]:
c07 = pyco2.sys(df07['ALKALI'], df07['TCARBN'], 1, 2,
               salinity=df07['CTDSAL'], temperature=df07['CTDTMP'], 
                pressure=df07['CTDPRS']);

/Users/tconnolly/opt/miniconda3/envs/ms263-24/lib/python3.11/site-packages/autograd/tracer.py:48: RuntimeWarning: invalid value encountered in sqrt
  return f_raw(*args, **kwargs)
/Users/tconnolly/opt/miniconda3/envs/ms263-24/lib/python3.11/site-packages/PyCO2SYS/equilibria/p1atm.py:99: RuntimeWarning: invalid value encountered in sqrt
  lnKF = 1590.2 / TempK - 12.641 + 1.525 * IonS**0.5
/Users/tconnolly/opt/miniconda3/envs/ms263-24/lib/python3.11/site-packages/PyCO2SYS/equilibria/p1atm.py:577: RuntimeWarning: overflow encountered in power
  K1 = 10.0 ** -(pK1)
/Users/tconnolly/opt/miniconda3/envs/ms263-24/lib/python3.11/site-packages/PyCO2SYS/equilibria/p1atm.py:583: RuntimeWarning: overflow encountered in power
  K2 = 10.0 ** -(pK2)
/Users/tconnolly/opt/miniconda3/envs/ms263-24/lib/python3.11/site-packages/PyCO2SYS/equilibria/p1atm.py:603: RuntimeWarning: overflow encountered in power
  K1 = 10.0**-pK1
/Users/tconnolly/opt/miniconda3/envs/ms263-24/lib/python3.11/site-packages/PyCO2SY

In [6]:
c13 = pyco2.sys(df13['ALKALI'], df13['TCARBN'], 1, 2,
               salinity=df13['CTDSAL'], temperature=df13['CTDTMP'], 
                pressure=df13['CTDPRS']);

### Multivariate regression tutorial: aragonite saturation state

Using data from the West Coast Ocean Acidification (WCOA) cruise, create two different multiple linear regression models to calculate aragonite saturation state (OmegaA) between 30 and 300 dbar as a function of more commonly observed variables.

#### Model 1: Temperature, salinity, pressure, dissolved oxygen and nitrate:

* Temperature
* Salinity
* Pressure
* Oxygen
* Nitrate

$$\hat{\Omega}_A = c_{0} + c_{1} \times T + c_{2} \times S + c_{3} \times p + c_{4} \times O_2 + c_{5} \times N$$

a. (*in class*) Create a 1-D array for the response variable, `y`. Which variable is the response variable in this case?

In [7]:
c07

{'par1': array([2350.1, 2264. , 2240.4, ..., 2281.8, 2276.4, 2265.5]),
 'par1_type': array([1, 1, 1, ..., 1, 1, 1]),
 'par2': array([   nan,    nan,    nan, ..., 2241.9, 2229.2, 2208. ]),
 'par2_type': array([2, 2, 2, ..., 2, 2, 2]),
 'alkalinity': array([2350.1, 2264. , 2240.4, ..., 2281.8, 2276.4, 2265.5]),
 'dic': array([   nan,    nan,    nan, ..., 2241.9, 2229.2, 2208. ]),
 'opt_k_bisulfate': 1,
 'opt_k_carbonic': 16,
 'opt_k_fluoride': 1,
 'opt_total_borate': 1,
 'opt_gas_constant': 3,
 'opt_pH_scale': 1,
 'opt_buffers_mode': 1,
 'salinity': array([12623.22, 12521.02, 12353.35, ...,    34.46,    34.39,    34.24]),
 'temperature': array([ 1.598,  1.729,  1.936, ..., 11.548, 12.256, 12.309]),
 'pressure': array([2946.5, 2502.2, 1996.8, ...,  175.6,  150.6,  130.4]),
 'total_ammonia': 0.0,
 'total_borate': array([149927.78725714, 148713.94325714, 146722.50271429, ...,
           409.28634286,    408.45494286,    406.67337143]),
 'total_calcium': array([3709268.16969029, 3679237.2261

In [8]:
# create a subset

ii = ((df07['CTDPRS'] >= 30) & (df07['CTDPRS'] <= 300) &
      (df07['NITRAT_FLAG_W'] ==2) & (df07['PHSPHT_FLAG_W'] ==2) &
      (df07['CTDOXY_FLAG_W'] == 2) & (df07['CTDSAL_FLAG_W'] ==2) &
      (df07['TCARBN_FLAG_W'] == 2) & (df07['ALKALI_FLAG_W'] == 2))

b. (*in class*) Solve for the coefficients $\vec{c}$ in the least squares problem:

$$ \hat{y} = \textbf{X}\vec{c} $$ 

Create a 2-D array `X` that contains a column of all ones, and additional columns containing the explanatory variables. This 2-D array is called the "design matrix" and should have six columns. What are the explanatory variables in this case? 

* Approach: use `np.ones()` to create a 2-D array of correct size, then fill in the columns.

c. (*in class*) Use `np.linalg.lstsq` to compute the set of coefficients, `c`.

Alternative solution using matrix multiplication:

$$\hat{c} = (\textbf{X}^T\textbf{X})^{-1}\textbf{X}^T\vec{y}$$

d. (*in class*) Use `np.matmul` to compute the modeled values `yhat` (students do)

e. (*in class*) Plot model vs. observations. (exercise)

f. (*in class*) Plot residuals vs. observations (exercise)

g. (*in class*) Use `statsmodels` to get a complete summary of regression statistics.

Condition number - this tells you how much a small change to the matrix $\textbf{X}$ will change the solution to the least squares problem $\hat{c} = (\textbf{X}^T\textbf{X})^{-1}\textbf{X}^T\vec{y}$. It is a measure of the extent to which the columns in $\textbf{X}$ are linearly independent and closely scaled in magnitude.

An illustrative example with a made-up matrix

h. (*in class*) Alternate approach using `statsmodels` formulas.

#### Alternative: Pingouin

In [94]:
pg.linear_regression(df07sub[['CTDTMP', 'CTDSAL', 'CTDPRS', 'CTDOXY', 'NITRAT']], df07sub['OmegaA'])

,names,coef,se,T,pval,r2,adj_r2,CI[2.5%],CI[97.5%]
0,Intercept,-2.757914,0.576098,-4.787228,1.991778e-06,0.972587,0.972427,-3.888645,-1.627183
1,CTDTMP,0.055835,0.005241,10.654495,5.628490e-25,0.972587,0.972427,0.045549,0.066121
2,CTDSAL,0.097123,0.019071,5.092696,4.343110e-07,0.972587,0.972427,0.059691,0.134554
3,CTDPRS,0.000875,0.000061,14.440292,1.836967e-42,0.972587,0.972427,0.000756,0.000994
4,CTDOXY,0.003308,0.000123,26.790351,2.623827e-115,0.972587,0.972427,0.003066,0.003551
5,NITRAT,-0.022500,0.001795,-12.533109,3.319621e-33,0.972587,0.972427,-0.026024,-0.018977


Check for multiple collinearity

### Application of final model for aragonite saturation state

In this section, data from the 2007 WCOA cruise are used to model aragonite saturation state following the model proposed by Juranek et al. (2009). An important question is whether this model can predict aragonite saturation state in different years. This model is can be tested using an independent data set from the 2013 WCOA cruise.

#### Final model: Dissolved oxygen, and the interaction between oxygen and temperature (subtracting constant reference values):

$ \hat{\Omega}_A = a_0 + a_1 \times (O - Oref) + a_2 \times (O - Oref) \times (T - Tref) $

a. (Exercise) Calculate the coefficients $a_0$, $a_1$ and $a_2$ using either the design matrix approach or `statsmodels` formulas.

In [ ]:
# insert code here

b. (Exercise) Calculate the root mean squared error for this  model.

In [ ]:
# insert code here

c. (Exercise) Plot the residuals vs. the observations.

In [ ]:
# insert code here

d. (Exercise) Use the coefficients calculated in part a to compute predicted aragonite saturation state for the 2013 cruise. Use the subset of 2013 data between 30 dbar and 300 dbar. 

In [ ]:
# insert code here

e. (Exercise) Calculate the root mean squared error (RMSE) between the model prediction and aragonite saturation state observations during 2013. Describe how this RMSE value compares with the RMSE calculated for the 2007 observations.

In [ ]:
# insert code here

*insert text here*

f. (Exercise)  Make a plot of the residuals vs. observations during 2013. Comment on whether there are any biases in the model.

In [ ]:
# insert code here

*insert text here*

g. (Exercise) In a paragraph, compare the two different regressions (model 1 and the final model), commenting on:
  * General applicability of the model equations
  * Statistical significance
  * Multiple co-linearity
  * The potential for numerical errors
  * How well the model represents aragonite saturation state in different years
  * Your scientific interpretation

*insert text here*